# Investing with 'FACTOR INVESTING'

## What is the CAPM?

**Capital Asset Pricing Model** CAPM is a financial model to evaluate the expected return of an asset. It provides a framework for understanding the relationship between the asset's risk and its expected return.

- CAPM considers market risk to be the primary determinant of an asset's expected return.
- Market risk is measured by beta, which represents the asset's sensitivity to market movements.
- The CAPM formula is: E(X) = risk free rate + Beta * (Market Return - Risk Free Rate)
- The **risk free rate** represents the return on a risk free investment, for example; government bonds.
- The CAPM model helps investors determine if the asset value is undervalued or overvalued. 







## What is the Fama-French Three-Factor Model

**The Fama-French Three Factor Model** is an extension of the CAPM. This model tries to explain expected returns of an asset by adding three more additional factors.

**Key Factors of Fama-French**
- Market Risk
- Size - Measures the difference in returns between small-cap and large-cap stocks
- Value - Measures the difference in returns betweeen high book-to-market ratio and low book-to-market ratio stocks

Fama French Model: E(X) = risk free rate + Beta * (Market Return - risk free rate) + size + value

By using the additional three factors the Fama-French model aims to better explain the cross-sectional variation in asset returns and provide a more comprehensive approach to asset pricing beyond CAPM's market risk consideration. 


## Cap - Weighted Benchmarks

Default approach is to use a **Cap-Weighted Index** as a *Benchmark* for **Active** or **Passive** managers.

But Cap-Weighted Indices are very inefficient. They tend to be **heavily concentrated** poorly diversified portfolios.

CW Indicies Provide an inefficient diversification of unrewarded and specific risks.

**Smart(Weighted) Benchmarks** have been introduced to fix the problem.

For example: 
- Equally-Weighted Benchmarks
- Minimum Variance Benchmarks
- Risk Parity Benchmarks

In [1]:
import pandas as pd 
brka_d = pd.read_csv('data/brka_d_ret.csv', parse_dates = True, index_col = 0)
brka_d.head()

,BRKA
DATE,
1990-01-02,-0.005764
1990-01-03,0.000000
1990-01-04,0.005797
1990-01-05,-0.005764
1990-01-08,0.000000


Next, we need to convert these to monthly returns. The simplest way to do so is by using the `.resample` method, which allows you to run an aggregation function on each group of returns in a time series. We'll give it the grouping rule of 'M' which means _monthly_ (consult the `pandas`) documentation for other codes)

We want to compound the returns, and we already have the `compound` function in our toolkit, so let's load that up now, and then apply it to the daily returns.

In [4]:
import risk_kit as rk

%load_ext autoreload
%autoreload 2

brka_m = brka_d.resample('M').apply(rk.compound).to_period('M')
brka_m.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,BRKA
DATE,
1990-01,-0.140634
1990-02,-0.030852
1990-03,-0.069204
1990-04,-0.003717
1990-05,0.067164


In [5]:
brka_m.to_csv('brka_m.csv')

In [7]:
fff = rk.get_fff_returns()
fff.head()

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031


Next, we need to decompose the observed BRKA 1990-May 2012 as in Ang(2014) into the portion that's due to the market and the rest that is not due to the market, using the CAPM as the explanatory model.

i.e.

$$ R_{brka,t} - R_{f,t} = \alpha + \beta(R_{mkt,t} - R_{f,t}) + \epsilon_t $$

We can use the `stats.api` for the linear regression as follows:

In [8]:
import statsmodels.api as sm 
import numpy as np

brka_excess = brka_m["1990":"2012-05"] - fff.loc['1990':'2012-05', ['RF']].values
mkt_excess = fff.loc['1990': '2012-05', ['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var['Constant'] = 1
lm = sm.OLS(brka_excess, exp_var).fit()

In [9]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BRKA   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     48.45
Date:                Sat, 22 Jul 2023   Prob (F-statistic):           2.62e-11
Time:                        21:44:20   Log-Likelihood:                 388.47
No. Observations:                 269   AIC:                            -772.9
Df Residuals:                     267   BIC:                            -765.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5402      0.078      6.961      0.000       0.387       0.693
Constant       0.0061      0.004      1.744      0.082      -0.001       0.013
==============================================================================
Omnibus:                       45.698   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.573
Skew:                           0.825   Prob(JB):                     5.33e-23
Kurtosis:                       5.535   Cond. No.                         22.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""